<a href="https://colab.research.google.com/github/fiorellaguillen/CASA0025/blob/main/PRACTICE_FOR_SQL_EXAM2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**INSTALL DUCKDB AND CONFIGURE IT**

In [1]:
%pip install duckdb duckdb-engine jupysql
%pip install leafmap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.8/192.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.7/213.7 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 765.5/765.5 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.2/194.2 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/7

In [2]:
import duckdb
import pandas as pd
import leafmap


In [3]:
# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [4]:
%%sql
duckdb:///:memory:

INSTALL spatial;
LOAD spatial;

INSTALL httpfs;
LOAD httpfs;

,Success


**LOAD DATA**

In [5]:
url = "https://open.gishub.org/data/duckdb/nyc_data.db.zip"
leafmap.download_file(url, unzip=True)

Downloading...
From: https://open.gishub.org/data/duckdb/nyc_data.db.zip
To: /content/nyc_data.db.zip
100%|██████████| 8.60M/8.60M [00:00<00:00, 93.6MB/s]


Extracting files...


'/content/nyc_data.db.zip'

In case you have a db file, first you connect to this database AND install spatial extension on this db

In [6]:
%%sql
duckdb:///nyc_data.db

INSTALL spatial;
LOAD spatial;


,Success


In [7]:
#Show all tables available on my db file

%%sql

SELECT * FROM ( SHOW TABLES);

,name
0,nyc_census_blocks
1,nyc_homicides
2,nyc_neighborhoods
3,nyc_streets
4,nyc_subway_stations


In [9]:
#get number of rows
%%sql

SELECT COUNT(*) FROM nyc_streets;


,count_star()
0,3982


In [10]:
%%sql

SELECT * FROM nyc_census_blocks
LIMIT 5

,BLKID,POPN_TOTAL,POPN_WHITE,POPN_BLACK,POPN_NATIV,POPN_ASIAN,POPN_OTHER,BORONAME,geom
0,360850009001000,97,51,32,1,5,8,Staten Island,"[5, 4, 184, 0, 0, 0, 0, 0, 55, 3, 13, 73, 151,..."
1,360850020011000,66,52,2,0,7,5,Staten Island,"[5, 4, 136, 0, 0, 0, 0, 0, 178, 58, 13, 73, 72..."
2,360850040001000,62,14,18,2,25,3,Staten Island,"[5, 4, 120, 0, 0, 0, 0, 0, 82, 227, 12, 73, 55..."
3,360850074001000,137,92,12,0,13,20,Staten Island,"[5, 4, 184, 0, 0, 0, 0, 0, 204, 85, 13, 73, 10..."
4,360850096011000,289,230,0,0,32,27,Staten Island,"[5, 4, 89, 0, 0, 0, 0, 0, 107, 247, 12, 73, 7,..."


In [18]:
#get number of total population

%%sql

SELECT SUM(POPN_TOTAL) / SUM(ST_AREA(GEOM)) *1000000  AS POP_DENSITY_PER_KM2
FROM nyc_census_blocks;


,POP_DENSITY_PER_KM2
0,10411.822814


In [22]:
%%sql

DROP TABLE IF EXISTS census;

CREATE TABLE census AS
SELECT * FROM nyc_census_blocks;

SELECT * FROM census LIMIT 5;

,BLKID,POPN_TOTAL,POPN_WHITE,POPN_BLACK,POPN_NATIV,POPN_ASIAN,POPN_OTHER,BORONAME,geom
0,360850009001000,97,51,32,1,5,8,Staten Island,"[5, 4, 184, 0, 0, 0, 0, 0, 55, 3, 13, 73, 151,..."
1,360850020011000,66,52,2,0,7,5,Staten Island,"[5, 4, 136, 0, 0, 0, 0, 0, 178, 58, 13, 73, 72..."
2,360850040001000,62,14,18,2,25,3,Staten Island,"[5, 4, 120, 0, 0, 0, 0, 0, 82, 227, 12, 73, 55..."
3,360850074001000,137,92,12,0,13,20,Staten Island,"[5, 4, 184, 0, 0, 0, 0, 0, 204, 85, 13, 73, 10..."
4,360850096011000,289,230,0,0,32,27,Staten Island,"[5, 4, 89, 0, 0, 0, 0, 0, 107, 247, 12, 73, 7,..."


In [30]:
#ADD A NEW COLUMN TO CENSUS WITH THE AREA OF GEOM

%%sql
ALTER TABLE census ADD COLUMN area DOUBLE;

UPDATE census SET area = ST_AREA(GEOM);

SELECT * FROM census LIMIT 5;


,BLKID,POPN_TOTAL,POPN_WHITE,POPN_BLACK,POPN_NATIV,POPN_ASIAN,POPN_OTHER,BORONAME,geom,area
0,360850009001000,97,51,32,1,5,8,Staten Island,"[5, 4, 184, 0, 0, 0, 0, 0, 55, 3, 13, 73, 151,...",22708.351682
1,360850020011000,66,52,2,0,7,5,Staten Island,"[5, 4, 136, 0, 0, 0, 0, 0, 178, 58, 13, 73, 72...",10306.106296
2,360850040001000,62,14,18,2,25,3,Staten Island,"[5, 4, 120, 0, 0, 0, 0, 0, 82, 227, 12, 73, 55...",13118.262298
3,360850074001000,137,92,12,0,13,20,Staten Island,"[5, 4, 184, 0, 0, 0, 0, 0, 204, 85, 13, 73, 10...",12912.908799
4,360850096011000,289,230,0,0,32,27,Staten Island,"[5, 4, 89, 0, 0, 0, 0, 0, 107, 247, 12, 73, 7,...",59455.608556


In [ ]:
#get name of columns, type of variables and more info

%%sql
SELECT * FROM (DESCRIBE cities);


,column_name,column_type,null,key,default,extra
0,id,BIGINT,YES,None,None,None
1,name,VARCHAR,YES,None,None,None
2,country,VARCHAR,YES,None,None,None
3,latitude,DOUBLE,YES,None,None,None
4,longitude,DOUBLE,YES,None,None,None
5,population,BIGINT,YES,None,None,None


In [44]:
# prompt: sql query to get boroname and the area of all blkid that belong to the same boroname
%%sql
SELECT boroname, SUM(area)/1000 AS total_area_sqkm
FROM census
GROUP BY boroname
ORDER BY total_area_sqkm desc
LIMIT 5;


,BORONAME,total_area_sqkm
0,Queens,283194.473614
1,Brooklyn,181927.497678
2,Staten Island,150758.328111
3,The Bronx,110196.022907
4,Manhattan,59091.860626
